In [1]:
# read saved results (tables) of old runs
# combine multiple runs
# make significance plots

import sys
import pandas as pd
import numpy as np
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
import pandas as pd
import core_tools as ct
import core_calib as cal
from scipy import stats


In [2]:
main_path = './Data manipulation/results'
# exp_names = ["1692118038_T10", "1692121657_T50", "1692130088_T100", "1692161204_O50", "1692172225_O100"] # , "1692107939_Z8T0"
# exp_names = ["1692351821_RFseed"]
exp_names = ["1698220976_opt_test_brier", "1698221678_opt_test_accuracy"]
exp_comp = "_ACC"

In [3]:
# to combine Calib and OOB runs in one CD plot
tables_all = {}
for exp in exp_names:
    path = f"{main_path}/{exp}"
    dict_data, tables, params = ct.read_results(path)
    if len(tables_all) == 0:
        tables_all = tables
    else: # combine tables
        for table_key in tables.keys():
            col_names = np.array((tables[table_key].columns))
            col_names += exp_comp
            tables[table_key].columns = col_names
            # print(tables[table_key])
            # print(f"key {table_key} col_names", col_names)
            tables_all[table_key] = pd.concat([tables_all[table_key], tables[table_key]], axis=1)
            # print(tables_all[table_key])

# tables_all = ct.add_rank_mean(tables_all)
tables_all["brier"]

# ct.res_statistics(tables_all, params["metrics"], ".")

,RF_d,RF_opt,RF_large,Platt,ISO,Beta,CRF,VA,tlr,Rank,...,RF_opt_ACC,RF_large_ACC,Platt_ACC,ISO_ACC,Beta_ACC,CRF_ACC,VA_ACC,tlr_ACC,Rank_ACC,Elkan_ACC
Data,,,,,,,,,,,,,,,,,,,,,
cm1,0.089384,0.085856,0.085684,0.087197,0.090606,0.087077,0.087496,0.091308,0.097692,0.093660,...,0.085566,0.085684,0.087029,0.090181,0.086751,0.087273,0.091932,0.098337,0.092068,0.219264
datatrieve,0.078904,0.080010,0.077905,0.086588,0.101139,0.109099,0.078461,0.098773,0.084738,0.081067,...,0.084086,0.077905,0.088042,0.098438,0.117242,0.077259,0.096065,0.084646,0.081881,0.232169
kc1_class_level_defectiveornot,0.179282,0.170893,0.168537,0.176773,0.212804,0.194227,0.179705,0.188395,0.190726,0.221542,...,0.178255,0.168537,0.177852,0.213818,0.191957,0.177693,0.186810,0.189080,0.220660,0.280019
kc1,0.105900,0.107887,0.102887,0.110099,0.112009,0.109044,0.108922,0.110472,0.130085,0.115971,...,0.107860,0.102887,0.109110,0.110454,0.108498,0.108036,0.109255,0.127065,0.113259,0.242629
kc2,0.122482,0.117333,0.119166,0.120073,0.125944,0.120826,0.118356,0.121700,0.131819,0.126887,...,0.116568,0.119166,0.121363,0.128333,0.122616,0.119687,0.123756,0.131193,0.129927,0.250715
kc3,0.076874,0.075909,0.073825,0.079295,0.083251,0.079277,0.078003,0.081106,0.085124,0.081570,...,0.076030,0.073825,0.077445,0.082512,0.077067,0.078480,0.079975,0.084962,0.082035,0.188447
pc1,0.051842,0.052759,0.049938,0.055233,0.056906,0.055464,0.053263,0.055282,0.062280,0.056500,...,0.053495,0.049938,0.054801,0.056731,0.055307,0.053519,0.054757,0.063701,0.055909,0.147895
spect,0.129725,0.125176,0.127147,0.128215,0.140123,0.129855,0.125986,0.132297,0.138631,0.160732,...,0.126111,0.127147,0.129285,0.143629,0.132088,0.126719,0.135409,0.135748,0.160294,0.164975
spectf,0.137301,0.131938,0.127215,0.138907,0.155685,0.143612,0.141245,0.144887,0.146755,0.164147,...,0.131523,0.127215,0.138566,0.154019,0.139902,0.146169,0.140392,0.150098,0.153738,0.169096


In [4]:

def stat_test_binary(test_table, method, metric, log=False):

    # Create two sample arrays
    Normal = np.array(test_table.iloc[:, 0])
    Modified = np.array(test_table.iloc[:, 1])

    result = Normal - Modified # for loss comparison
    # result = Modified - Normal # for acc comparison
    if metric == "acc":
        result[result > 0] = 0
        result[result < 0] = 1
    else:
        result[result > 0] = 1
        result[result < 0] = 0
    n_trials = len(result) 
    observed_successes = int(sum(result))

    if log:
        print(f"{metric} {method} observed_successes", observed_successes)

    result = stats.binomtest(observed_successes, n=n_trials, p=0.5, alternative='less')
    p_value = result.pvalue

    if log:
        print("p_value", p_value)

    alpha = 0.05

    if p_value < alpha:
        if log:
            print("Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.")
        return 1
    else:
        if log:
            print("Fail to reject the null hypothesis. There is no significant difference.")
        return 0

def stat_test_ttest(test_table, method, metric, log=False):

    if metric == "acc":
        statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='less')
    else:
        statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='greater')


    if log:
        print("p_value", p_value)
    alpha = 0.05

    if p_value < alpha:
        if log:
            # print("Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.")
            print("The change creates significant improvment")
        return '\ding{51}'
    else:
        if metric == "acc":
            statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='greater')
        else:
            statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='less')

        if p_value < alpha:
            if log:
                print("The change creates significant decline.")
            return '\ding{55}'
        else:
            if log:    
                print("Fail to reject the null hypothesis. There is no significant difference.")
        return "  "



In [5]:
calib_methods = [   "RF_d", "RF_opt", "RF_large",
                    "Platt", "ISO", "Beta", "CRF", "VA",
                    "tlr", "Rank", #"Elkan",                      
                ]

metrics =  ["acc", "brier", "logloss", "ece"]

res_table = []
for method in calib_methods:
    res_array = [method]
    for metric in metrics:
        test_table = tables_all[metric][[method, method + exp_comp]]

        # if method == "RF_opt" and metric == "brier":
        #     print(test_table)

        res_array.append(stat_test_ttest(test_table, method, metric))
    res_table.append(res_array)

res_table = np.array(res_table)

res_table = pd.DataFrame(res_table, columns=["calib_methods"]+ metrics)
res_table.set_index('calib_methods', inplace=True)
res_table.to_latex(f"./comp_stat_test.txt")
res_table

/var/folders/gm/nlbfzh25075c0jwn9khghrfw0000gn/T/ipykernel_69634/1986785805.py:24: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  res_table.to_latex(f"./comp_stat_test.txt")


,acc,brier,logloss,ece
calib_methods,,,,
RF_d,,,,
RF_opt,,,\ding{55},\ding{55}
RF_large,,,,
Platt,,,,
ISO,,,,
Beta,\ding{55},,,
CRF,,,,\ding{55}
VA,,,,
tlr,,,,
